In [1]:
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from hiv_patient import HIVPatient
from sklearn.ensemble import ExtraTreesRegressor 

In [117]:
def greedy_action(tree, state, env):
    rew = []
    for j in range(4):
        input_a = np.zeros((1, 8))
        input_a[:, :6] = state
        input_a[:, 6:] = env.action_set[j]
        rew.append(tree.predict(input_a))
 
    return np.argmax(rew)

In [118]:
# Replay buffer class
class ReplayBuffer:
    def __init__(self, 
                 env,
                 gamma=0.9,
                 epsilon=0.15,
                 nb_actions=4,
                 nb_patient=30,
                 duration=200):
        

        self.env = env
        self.gamma = gamma 
        self.epsilon = epsilon
        self.nb_actions = nb_actions
        self.nb_patients = nb_patient
        self.duration = duration
        self.data = np.zeros((self.duration * self.nb_patients, 15))


        print('Initializing replay buffer RB_0:')
        for j in tqdm(range(self.nb_patients)):
            self.env.reset()
            for k in range(self.duration):
                s = self.env.state()
                idx_a = np.random.randint(self.nb_actions)
                a = self.env.action_set[idx_a]
                s_, r, _, _ = self.env.step(idx_a)

                # state size 6,  action size 2,  reward size 1
                self.data[j*self.duration + k, : 6] = s # state
                self.data[j*self.duration + k, 6:8] = a # action
                self.data[j*self.duration + k, 8] = r # reward
                self.data[j*self.duration + k, 9: ]  = s_ # new state


    def update_set(self, tree, data):
        self.data_new = np.zeros((self.duration * self.nb_patients, 15))
        for j in range(self.nb_patients):
            self.env.reset()
            for k in range(self.duration):
                # epsilon greedy
                s = self.env.state() 
                if np.random.rand() < self.epsilon:
                    idx_a = np.random.randint(self.nb_actions)
                else :
                    idx_a = greedy_action(tree, s, self.env)
                
                s_, r, _, _ = self.env.step(idx_a)
                a = self.env.action_set[idx_a]
                self.data_new[j*self.duration + k, : 6] = s # state
                self.data_new[j*self.duration + k, 6:8] = a # action
                self.data_new[j*self.duration + k, 8] = r # reward
                self.data_new[j*self.duration + k, 9: ]  = s_ # new state
        update_data = np.concatenate([data, self.data_new])
        
        return update_data

        
        

    def __len__(self):
        return len(self.data)

In [119]:
class ExtraTree:
    def __init__(self, 
                 config,
                 env,
                 nb_epoch) -> None:
        self.env = env
        self.nb_epoch = nb_epoch
        self.duration = config['duration']
        self.nb_patient = config['nb_patients']
        self.gamma = config['gamma']
        self.buffer = ReplayBuffer(env,
                                 gamma=self.gamma,
                                 epsilon=0.15,
                                 nb_actions=len(self.env.action_set),
                                 duration=self.duration, 
                                 nb_patient=self.nb_patient)
        self.data = self.buffer.data
        self.Qtree = ExtraTreesRegressor()
    
    def train(self):

        # First step
        X, y = self.data[:, :8], self.data[:, 8]
        self.Qtree.fit(X,y)

        for epoch in range(self.nb_epoch):
            print('Epoch: ', epoch)
            print('-------')
            X_ = np.zeros((self.data.shape[0], 8))
            X_[:, :6]= self.data[:, 9:] # s prime
            y_res = np.zeros((self.data.shape[0], 4))
            
            # We loop on all possible actions 
            for idx_a in range(4):
                a = self.env.action_set[idx_a]
                X_[:, 6:] = a
                y_pred = self.Qtree.predict(X_)
                y_res[:, idx_a] = y_pred

            # We take the argmax and update our training set
            idx_max = np.argmax(y_res, axis=1)
            print(np.take_along_axis(y_res, idx_max[:,None], axis=1).shape)
            self.data[:,8] = self.data[:, 8] + self.gamma * np.take_along_axis(y_res, idx_max[:,None], axis=1)[:, 0]

            #We update the training set
            self.data = self.buffer.update_set(self.Qtree, self.data)

            # fitting the tree with the new training set
            X, y = self.data[:, :8], self.data[:, 8]
            self.Qtree.fit(X,y)

In [120]:
patient = HIVPatient()
config_test = {'gamma': 0.99,
               'nb_patients': 5,
               'duration': 10 }


Tree_test = ExtraTree(config=config_test,
                      env=patient,
                      nb_epoch= 5)

Initializing replay buffer RB_0:


100%|██████████| 5/5 [00:01<00:00,  3.02it/s]


In [121]:
Tree_test.train()

Epoch:  0
-------
(50, 1)
Epoch:  1
-------
(100, 1)
Epoch:  2
-------
(150, 1)
Epoch:  3
-------
(200, 1)
Epoch:  4
-------
(250, 1)
